In [22]:

# import library
import pandas as pd
import numpy as np
import requests
import io

try:
    from StringIO import StringIO
except ImportError:
    
    from io import StringIO
    from io import BytesIO
from decimal import Decimal
from datetime import datetime
google_sheet_url = 'https://docs.google.com/spreadsheets/d/1ngEPTI_t5CpKODrB5foQQvps6QOcjCpM98_Y0oCEcBY/export?format=xlsx'
r = requests.get(google_sheet_url)
data = r.content
df = pd.read_excel(BytesIO(data),sheet_name='new Staff')

df['CODE'] = df['CODE'].fillna(0)
df['CODE'] = df['CODE'].astype(int)
df['POINT'] = df['POINT'].fillna(0)
df['POINT'] = df['POINT'].astype(str)

df.head()


<ipython-input-22-a00b38777133>:18: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df = pd.read_excel(BytesIO(data),sheet_name='new Staff')


,POINT,CODE,NAME,TM 500,STD 500,STD 250,FCM 500,Total
0,161,428167,NaN,0,1,0,0,1


In [21]:
missing_list = []
missind_customer_list = []
start_date = '2021-03-01'
month = 3
year = 2021
session_id = 1
product_obj = ICustomerTypeWiseProductDiscount.objects.filter(customer_type_id=2, product__is_active=True).order_by('product__display_ordinal')
product_obj_list = list(
    product_obj.values_list('product_id', 'product__group', 'product__group__name',
                                       'product__name', 'product__short_name', 'product__code',
                                       'product__unit',
                                       'product__unit__display_name', 'product__description',
                                       'product__quantity', 'discounted_price', 'product__gst_percent','product__color'))
product_obj_column = ['product_id', 'product_group_id', 'product_group_name', 'product_name',
                             'product_short_name',
                             'product_code', 'unit_id', 'unit_name', 'description', 'product_quantity',
                             'product_mrp',
                             'gst_percent','color']
product_df = pd.DataFrame(product_obj_list, columns=product_obj_column)
product_list = product_df.to_dict('r')
product_dict = {}
for index, row in df.iterrows():
    print(row['POINT'])
    row['POINT'] = '161'
    print(row['POINT'])
    print('crossed - {} - booth code -{}'.format(index, row['POINT']))
    if not Business.objects.filter(code=row['POINT']).exists():
        missing_list.append(row['POINT'])
        print('{} code is not available'.format(row['POINT']))
        continue;
    if not row['Total'] == 0:
        print('Value greated than zero') 
        if ICustomer.objects.filter(customer_code='EXEMP' + str(row['CODE'])).exists():
#         if ICustomer.objects.filter(customer_code=str(row['CODE'])).exists():
            business_obj = Business.objects.get(code=row['POINT'])
            icustomer_obj = ICustomer.objects.get(customer_code='EXEMP' + str(row['CODE']))
#             icustomer_obj = ICustomer.objects.get(customer_code=row['CODE'])
            icustomer_obj.business_id = business_obj.id
            icustomer_obj.save()
            icustomer_id = icustomer_obj.id
            route_id = RouteBusinessMap.objects.get(business_id=business_obj.id,
                                                    route__session_id=session_id).route_id
            total_cost_per_session = 0
            total_cost_per_session_for_month = 0
            sale_group_obj = ICustomerSaleGroup(
                business_id=business_obj.id,
                icustomer_id=icustomer_id,
                date=start_date,
                session_id=session_id,
                route_id=route_id,
                zone_id=business_obj.zone.id,
                ordered_via_id=2,
                payment_status_id=1,  # paid
                sale_status_id=1,  # ordered
                total_cost=0,
                total_cost_for_month=0,
                ordered_by_id=1,
                modified_by_id=1,
                product_amount=0,
            )
            sale_group_obj.save()
            for product in product_list:
                if product['product_short_name'] in row.keys():
                    product_quantity = row[product['product_short_name']]
#                         product_quantity = Decimal(product_quantity)
                    if product_quantity != 0 and product_quantity != None:
                        sale_obj = ICustomerSale(
                            icustomer_sale_group_id=sale_group_obj.id,
                            product_id=product['product_id'],
                            count=product_quantity,
                            cost=product_quantity * product['product_mrp'],
                            cost_for_month=product_quantity * product['product_mrp'] * 31,
                            ordered_by_id=1,
                            modified_by_id=1
                        )

                        sale_obj.save()
                        product_dict[product['product_id']] = product_quantity
                        total_cost_per_session += product_quantity * product['product_mrp']
                        total_cost_per_session_for_month += product_quantity * product['product_mrp'] * 31

            sale_group_obj.total_cost = total_cost_per_session
            sale_group_obj.total_cost_for_month = total_cost_per_session_for_month
            sale_group_obj.product_amount = total_cost_per_session
            sale_group_obj.save()
            print('Customer order saved {}'.format(row['CODE']))
            if ICustomerMonthlyOrderTransaction.objects.filter(icustomer_id=icustomer_id, month=month, year=year).exists():
                icustomer_monthly_order_obj = ICustomerMonthlyOrderTransaction.objects.get(icustomer_id=icustomer_id, month=month, year=year)
                is_date_available = True
            else:
                icustomer_monthly_order_obj = ICustomerMonthlyOrderTransaction(icustomer_id=icustomer_id, month=month, year=year, transacted_date_time=datetime.now(), total_cost=0, created_by_id=1, milk_card_number=0)
                icustomer_monthly_order_obj.save()
                is_date_available = False

            icustomer_monthly_order_obj.total_cost += sale_group_obj.total_cost_for_month
            icustomer_monthly_order_obj.icustomer_sale_group.add(sale_group_obj.id)
            icustomer_milk_card_id_bank_obj = ICustomerMilkCarkNumberIdBank.objects.get(id=1)
            if is_date_available:
                last_count = icustomer_milk_card_id_bank_obj.last_milk_card_number
            else:
                last_count = icustomer_milk_card_id_bank_obj.last_milk_card_number + 1
            icustomer_monthly_order_obj.milk_card_number = str(last_count).zfill(6)
            icustomer_monthly_order_obj.save()
            # update_last_count_in_id_bank
            icustomer_milk_card_id_bank_obj.last_milk_card_number = last_count
            icustomer_milk_card_id_bank_obj.save()
            
#             serial number code
            if ICustomerCardSerialNumberIdBank.objects.filter(business_id=business_obj.id, month=month, year=year).exists():
                icustomer_serial_bank_id_obj = ICustomerCardSerialNumberIdBank.objects.get(business_id=business_obj.id, month=month, year=year)
            else:
                icustomer_serial_bank_id_obj = ICustomerCardSerialNumberIdBank(business_id=business_obj.id, month=month, year=year, counter_last_count=0, online_last_count=0)
                icustomer_serial_bank_id_obj.save()

            icustomer_serial_number_map = ICustomerSerialNumberMap(icustomer_id=icustomer_id,business_id=business_obj.id, month=month, year=year)
            serial_number = icustomer_serial_bank_id_obj.counter_last_count + 1
            icustomer_serial_number_map.serial_number = serial_number
            icustomer_serial_number_map.save()
            # update serail number in ID bank
            icustomer_serial_bank_id_obj.counter_last_count = serial_number
            icustomer_serial_bank_id_obj.online_last_count = serial_number
            icustomer_serial_bank_id_obj.save()
        else:
            print('Customer not exists {}'.format(row['CODE']))
            missind_customer_list.append(row['CODE'])

161
161
crossed - 0 - booth code -161
Value greated than zero
Customer not exists 428167


In [17]:
for i in missind_customer_list:
    print(i)
missing_list

[161.0]

In [37]:
ICustomerSaleGroup.objects.filter(icustomer__customer_type_id__in=[2,3], date__month=2, date__year=2021).delete()

(1680,
 {'main.ICustomerSale': 560,
  'main.ICustomerMonthlyOrderTransaction_icustomer_sale_group': 560,
  'main.CounterEmployeeTraceSaleGroupMap_icustomer_sale_group': 0,
  'main.ICustomerSaleGroupPaymentRequestMap_sale_group': 0,
  'main.ICustomerSaleGroup': 560})

In [34]:
# 'EXEMP' + str(row['CODE'])
# BusinessAgentMap.objects.filter(business__code=2657)
ICustomerSerialNumberMap.objects.filter(icustomer__customer_type_id__in=[2, 3]).delete()

(1023, {'main.ICustomerSerialNumberMap': 1023})

In [34]:
ICustomerSaleGroup.objects.filter(date__month=7).count()

0

In [18]:
def get_business_ids(route_id, session_id, date):
    # return ordered_business
#     RouteBusinessMap.objects.filter(route_id=route_id).values_list('business_id', flat=True)
    business_ids = [1660]
    ordered_business = list(
        SaleGroup.objects.filter(business_id__in=business_ids, date=date, session_id=session_id).values_list(
            'business_id', flat=True))
    selected_month = date
    ordered_business_for_icustomer = list(
        ICustomerSaleGroup.objects.filter(business_id__in=business_ids, date__month=selected_month.month,
                                          date__year=selected_month.year, session_id=session_id).values_list(
            'business_id', flat=True))
    return list(set(ordered_business + ordered_business_for_icustomer))

In [67]:
Route.objects.get(id=87).name

'W - ROUTE_MOR'

In [19]:
import datetime
date = datetime.datetime.now().date()
date += datetime.timedelta(days=1)
get_business_ids(106, 1, date)

[1660]

In [23]:
list(SaleGroup.objects.filter(business_id__in=[1660], date=date, session_id=1).values_list(
            'business_id', flat=True))
selected_month = date
ICustomerSaleGroup.objects.filter(business_id__in=[1660], date__month=selected_month.month,
                                          date__year=selected_month.year, session_id=1)

<QuerySet [<ICustomerSaleGroup: ICustomerSaleGroup object>, <ICustomerSaleGroup: ICustomerSaleGroup object>]>

In [52]:
ICustomer.objects.get(customer_code='EXEMP20091')

<ICustomer: 7626>

In [2]:
for index, row in df.iterrows():
    print('EXEMP' + str(row['CODE']))
    if ICustomer.objects.filter(customer_code='EXEMP' + str(row['CODE'])).exists():
#     if ICustomer.objects.filter(customer_code='EXEMP' + str(row['CODE'])).exists():
        print('yes')
    else:
        print('NO')

EXEMP10052
yes
EXEMP10037
yes
EXEMP10216
yes
EXEMP10110
yes
EXEMP10242
yes
EXEMP1722
yes
EXEMP615
yes
EXEMP10496
yes
EXEMP10236
yes
EXEMP1485
yes
EXEMP10142
yes
EXEMP10385
yes
EXEMP10368
yes
EXEMP10143
yes
EXEMP10370
yes
EXEMP10121
yes
EXEMP10078
yes
EXEMP10390
yes
EXEMP10332
yes
EXEMP10126
yes
EXEMP10144
yes
EXEMP10323
yes
EXEMP10146
yes
EXEMP10082
yes
EXEMP10059
yes
EXEMP10083
yes
EXEMP10284
yes
EXEMP10373
yes
EXEMP10560
yes
EXEMP10162
yes
EXEMP10226
yes
EXEMP10267
yes
EXEMP10253
yes
EXEMP10325
yes
EXEMP10206
yes
EXEMP10033
yes
EXEMP10302
yes
EXEMP10495
yes
EXEMP10016
yes
EXEMP10020
yes
EXEMP10024
yes
EXEMP10045
yes
EXEMP10084
yes
EXEMP10088
yes
EXEMP10097
yes
EXEMP10186
yes
EXEMP10200
yes
EXEMP10203
yes
EXEMP10211
yes
EXEMP10289
yes
EXEMP10301
yes
EXEMP10303
yes
EXEMP10307
yes
EXEMP10312
yes
EXEMP10314
yes
EXEMP10316
yes
EXEMP10317
yes
EXEMP10320
yes
EXEMP10331
yes
EXEMP10334
yes
EXEMP10335
yes
EXEMP10336
yes
EXEMP10337
yes
EXEMP10348
yes
EXEMP10404
yes
EXEMP10429
yes
EXEMP10475
yes

In [73]:
ICustomer.objects.filter(date__month=5).count()

FieldError: Cannot resolve keyword 'date' into field. Choices are: aadhar_document, aadhar_number, business, business_id, customer_code, customer_type, customer_type_id, date_of_birth, icustomerproductmap, icustomersalegroup, icustomersmartcard, icustomerwallet, id, is_business_approved, kyc_idproof_document_type, kyc_idproof_document_type_id, kyc_idproof_document_value, time_created, time_modified, user_profile, user_profile_id

In [5]:
date_list = []
for rout in RouteTrace.objects.filter(date__month=5).exclude(date='2020-05-01'):
    date_list.append(rout.date)
len(list(set(date_list)))

31

In [24]:
ICustomerSaleGroup.objects.filter(icustomer__customer_type_id=1, date__month=5).delete()

0

In [17]:
RouteTrace.objects.filter(date__month=5).exclude(date__in=['2020-05-01', '2020-05-02']).delete()

(0, {})

In [20]:
OverallIndentPerSession.objects.filter(date__month=5).exclude(date__in=['2020-05-01', '2020-05-02']).delete()

(58,
 {'main.OverallIndentPerSession_route_trace': 0,
  'main.OverallIndentPerSession': 58})

In [54]:
for icustomer in ICustomer.objects.filter():
    if ICustomerSerialNumberMap.objects.filter(month=9, year=2020, icustomer=icustomer).count() > 1:
        print(icustomer.customer_code)

CBE207656
CBE426873
CBE426871
CBE100061


In [1]:
for icustomer in ICustomer.objects.filter():
    if ICustomerSerialNumberMap.objects.filter(month=9, year=2020, icustomer=icustomer).count() > 1:
        for serial in ICustomerSerialNumberMap.objects.filter(month=9, year=2020, icustomer=icustomer):
            print(serial.time_created)
        print(icustomer.customer_code, icustomer.business.zone.name, icustomer.customer_type.name)

2020-08-29 08:58:10.761854+00:00
2020-08-29 08:57:02.806169+00:00
CBE207656 NORTH Customer
2020-08-29 09:23:54.685510+00:00
2020-08-29 08:49:55.658454+00:00
EXEMP1485 NORTH Ex - employee
2020-08-29 09:23:54.838698+00:00
2020-08-29 08:49:55.818419+00:00
EXEMP10385 NORTH Ex - employee
2020-08-29 09:23:55.156048+00:00
2020-08-29 08:49:56.189316+00:00
EXEMP10143 NORTH Ex - employee
2020-08-29 09:23:55.190008+00:00
2020-08-29 08:49:56.218441+00:00
EXEMP10370 NORTH Ex - employee
2020-08-29 09:23:55.281936+00:00
2020-08-29 08:49:56.272566+00:00
EXEMP10121 NORTH Ex - employee
2020-08-29 09:23:56.398999+00:00
2020-08-29 08:49:57.546490+00:00
EXEMP10390 NORTH Ex - employee
2020-08-29 09:23:56.609338+00:00
2020-08-29 08:49:57.760707+00:00
EXEMP10332 NORTH Ex - employee
2020-08-29 09:23:57.107240+00:00
2020-08-29 08:49:58.267901+00:00
EXEMP10126 NORTH Ex - employee
2020-08-29 09:23:57.157091+00:00
2020-08-29 08:49:58.326249+00:00
EXEMP10323 NORTH Ex - employee
2020-08-29 09:23:57.212418+00:00
2020

2020-08-29 09:23:55.083433+00:00
2020-08-29 08:49:56.094101+00:00
EXEMP10195 WEST Ex - employee
2020-08-29 09:23:55.108013+00:00
2020-08-29 08:49:56.122075+00:00
EXEMP10212 WEST Ex - employee
2020-08-29 09:23:55.332450+00:00
2020-08-29 08:49:56.435437+00:00
EXEMP11207 EAST Ex - employee
2020-08-29 09:23:55.358222+00:00
2020-08-29 08:49:56.491511+00:00
EXEMP10259 SOUTH Ex - employee
2020-08-29 09:23:55.696926+00:00
2020-08-29 08:49:56.844858+00:00
EXEMP10086 SOUTH Ex - employee
2020-08-29 09:23:55.721494+00:00
2020-08-29 08:49:56.880180+00:00
EXEMP10090 SOUTH Ex - employee
2020-08-29 09:23:55.770534+00:00
2020-08-29 08:49:56.929558+00:00
EXEMP10164 SOUTH Ex - employee
2020-08-29 09:23:55.800870+00:00
2020-08-29 08:49:56.955398+00:00
EXEMP10181 SOUTH Ex - employee
2020-08-29 09:23:55.850642+00:00
2020-08-29 08:49:57.011325+00:00
EXEMP10264 SOUTH Ex - employee
2020-08-29 09:23:55.907600+00:00
2020-08-29 08:49:57.059977+00:00
EXEMP10360 SOUTH Ex - employee
2020-08-29 09:23:55.933812+00:00


2020-08-29 09:23:57.574585+00:00
2020-08-29 08:49:58.744085+00:00
EXEMP20097 WEST Ex - employee
2020-08-27 06:01:08.037899+00:00
2020-08-27 06:04:39.168218+00:00
2020-08-27 06:04:53.409965+00:00
CBE426873 WEST Customer
2020-08-28 05:56:50.394697+00:00
2020-08-28 05:56:57.435766+00:00
CBE426871 WEST Customer
2020-08-29 09:23:50.379335+00:00
2020-08-29 08:49:52.061121+00:00
EXEMP10025 WEST Ex - employee
2020-08-29 09:23:50.639336+00:00
2020-08-29 08:49:52.323031+00:00
EXEMP10373 SOUTH Ex - employee
2020-08-29 09:23:50.548025+00:00
2020-08-29 08:49:52.239993+00:00
EXEMP10497 WEST Ex - employee
2020-08-29 09:23:50.692171+00:00
2020-08-29 08:49:52.370878+00:00
EXEMP10013 WEST Ex - employee
2020-08-29 09:23:50.766976+00:00
2020-08-29 08:49:52.462189+00:00
EXEMP10226 SOUTH Ex - employee
2020-08-29 09:23:50.913629+00:00
2020-08-29 08:49:52.570049+00:00
EXEMP10206 SOUTH Ex - employee
2020-08-29 09:23:51.076651+00:00
2020-08-29 08:49:52.735472+00:00
EXEMP11206 WEST Ex - employee
2020-08-29 09:23

In [38]:
unique_fields = ['icustomer', 'month', 'year']
duplicates = (
    ICustomerSerialNumberMap.objects.values(*unique_fields)
    .order_by()
    .annotate(max_id=Max('id'), count_id=Count('id'))
    .filter(count_id__gt=1)
)
duplicates.count()

238

In [39]:
for d in duplicates: 
    print(ICustomer.objects.get(id=d['icustomer']).customer_code)

EXEMP10259
EXEMP10058
EXEMP10078
EXEMP11205
EXEMP10692
EXEMP10303
EXEMP10566
EXEMP10336
EXEMP10328
EXEMP10141
EXEMP10253
EXEMP10110
EXEMP10134
EXEMP10267
EXEMP10690
EXEMP11203
EXEMP10475
EXEMP10232
EXEMP10247
EXEMP20094
EXEMP10025
EXEMP91
EXEMP10337
EXEMP10195
EXEMP10560
EXEMP10087
EXEMP10287
EXEMP427675
EXEMP10340
EXEMP2573
EXEMP1485
EXEMP10335
EXEMP2505
EXEMP10386
EXEMP10105
EXEMP10373
EXEMP10160
EXEMP10582
EXEMP10620
EXEMP10146
EXEMP10265
EXEMP10350
EXEMP10693
EXEMP10280
EXEMP10695
EXEMP10326
EXEMP10149
EXEMP10008
EXEMP10314
EXEMP10144
EXEMP10138
EXEMP10266
EXEMP10348
EXEMP10085
EXEMP198
EXEMP10222
EXEMP10689
EXEMP10216
EXEMP10694
EXEMP10037
EXEMP10142
EXEMP10180
EXEMP10185
EXEMP10262
EXEMP10342
EXEMP10184
EXEMP422
EXEMP10121
EXEMP2748
EXEMP10053
EXEMP10045
EXEMP10099
EXEMP10331
EXEMP10370
EXEMP10005
EXEMP10047
EXEMP10064
EXEMP10126
EXEMP10227
EXEMP10069
EXEMP10080
EXEMP427813
EXEMP10264
EXEMP426652
EXEMP10051
EXEMP10168
EXEMP10140
EXEMP10541
EXEMP10568
EXEMP10071
EXEMP10385
EXEMP10